# LTE Data Science Capstone Project
Develop a machine learning model to analyze and predict car accident severity


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
# Get Data
!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

df = pd.read_csv('Data-Collisions.csv')

--2020-09-11 14:20:37--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv’

100%[======================================>] 73,917,638  47.3MB/s   in 1.5s   

2020-09-11 14:20:39 (47.3 MB/s) - ‘Data-Collisions.csv’ saved [73917638/73917638]



/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# DOWN sample
from sklearn.utils import resample

# Separate majority and minority classes
df_maj = df[df.SEVERITYCODE == 1]
df_min = df[df.SEVERITYCODE == 2]

# Down sample minority class
df_maj_dnsampled = resample(df_maj, 
                            replace = False,     # sample with replacement
                            n_samples = 58188)    # to match minority class

# Combine majority class with down sampled  class
df_dnsampled = pd.concat([df_min, df_maj_dnsampled])
 
# Display new class counts
df_dnsampled.SEVERITYCODE.value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

In [4]:
# Down Sample 2
feature = df_dnsampled[['WEATHER', 'ROADCOND', 'LIGHTCOND']] #.values
y = df_dnsampled['SEVERITYCODE'].values


cleanup_nums = {'WEATHER': {"Clear": 11, 'Raining': 1, 'Overcast': 2, 'Unknown': 3,
                              'Snowing': 4, 'Other': 5, 'Fog/Smog/Smoke': 6, 'Sleet/Hail/Freezing Rain': 7,
                              'Blowing Sand/Dirt': 8, 'Severe Crosswind': 9, 'Partly Cloudy': 10},
                'ROADCOND': {"Dry": 11, "Wet": 1, "Unknown": 2, "Ice": 3,
                              "Snow/Slush": 4, "Other": 5, "Standing Water": 6,
                              'Sand/Mud/Dirt': 7, 'Oil': 8},
               'LIGHTCOND': {'Daylight': 11, 'Dark - Street Lights On': 1, 'Unknown': 2, 'Dusk': 3, 
                            'Dawn': 4, 'Dark - No Street Lights': 5, 'Dark - Street Lights Off': 6,
                            'Other': 7, 'Dark - Unknown Lighting':8}}

feature.replace(cleanup_nums, inplace=True)

X = feature

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:6517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


# Category Transformation: Find/Replace

In [5]:
X.replace(np.nan,11, inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [10]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[-1.18965992, -1.50762022,  0.7419481 ],
       [-1.41028904, -1.50762022,  0.7419481 ],
       [ 0.79600218,  0.68776244,  0.7419481 ],
       [ 0.79600218,  0.68776244,  0.7419481 ],
       [ 0.79600218,  0.68776244, -1.45202053]])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (81463, 3) (81463,)
Test set: (34913, 3) (34913,)


# K Nearest Neighbor

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

Ks = 20
mean_acc = np.zeros((Ks-1))
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train, y_train)
    yhat = neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

print("The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

The best accuracy was with 0.5564975796981068 with k= 17


In [19]:
neigh = KNeighborsClassifier(n_neighbors = 17).fit(X_train,y_train)
neigh

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=17, p=2,
           weights='uniform')

In [22]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

#KNN Calculations
yhat_knn = neigh.predict(X_test)

# jaccard
jaccard_knn = jaccard_similarity_score(y_test, yhat_knn)
print("KNN Jaccard index: ", jaccard_knn)

# f1_score
f1_score_knn = f1_score(y_test, yhat_knn, average='weighted')
print("KNN F1-score: ", f1_score_knn)

KNN Jaccard index:  0.5564975796981068
KNN F1-score:  0.5313474488839894


# Decision Tree

In [23]:
from sklearn.tree import DecisionTreeClassifier

Tree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
Tree.fit(X_train,y_train)
yhatTree = Tree.predict(X_test)

print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, yhatTree))

DecisionTrees's Accuracy:  0.5571277174691376


In [24]:
#Decision Tree Calculations
yhat_dt = Tree.predict(X_test)

# jaccard
jaccard_dt = jaccard_similarity_score(y_test, yhat_dt)
print("DT Jaccard index: ", jaccard_dt)

# f1_score
f1_score_dt = f1_score(y_test, yhat_dt, average='weighted')
print("DT F1-score: ", f1_score_dt)

DT Jaccard index:  0.5571277174691376
DT F1-score:  0.5217561355391311


# Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)

In [26]:
#Logistic Regression Calculations
yhat_lg = LR.predict(X_test)
yhat_lg_prob = LR.predict_proba(X_test)

# jaccard
jaccard_lg = jaccard_similarity_score(y_test, yhat_lg)
print("LR Jaccard index: ", jaccard_lg)

# f1_score
f1_score_lg = f1_score(y_test, yhat_lg, average='weighted')
print("LR F1-score: ", f1_score_lg)

LR Jaccard index:  0.5407727780482915
LR F1-score:  0.5334004908537546


# Report

| Algorithm          | Jaccard | F1-score | 
|--------------------|---------|----------|
| KNN                | 0.55649  | 0.53134        | 
| Decision Tree      | 0.55712       | 0.52175        | 
| LogisticRegression | 0.54077       | 0.5334       | 

##### On the basis the KNN method had a higher F1 score and only slightly lower Jaccard score compared to the Decision Tree, it is recommended the KNN method be used to determine accident severity.